In [1]:
import pandas as pd
import spacy
import re

In [3]:
df = pd.read_csv(r"C:\Users\Edris\Desktop\Python Programmeri ng\Stice\skadebelopp\skade belopp\data.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              7000 non-null   int64 
 1   Id                      7000 non-null   object
 2   Domstol                 7000 non-null   object
 3   Malnummer               6489 non-null   object
 4   Domsnummer              2208 non-null   object
 5   Avgorandedatum          6588 non-null   object
 6   Referat dom             6589 non-null   object
 7   Avdelning               6586 non-null   object
 8   Rubrik                  7000 non-null   object
 9   Lagrum                  6594 non-null   object
 10  Rattsfall               3899 non-null   object
 11  Referat                 7000 non-null   object
 12  Sammanfattning          7000 non-null   object
 13  Utfall                  7000 non-null   object
 14  Sokord                  7000 non-null   object
 15  Litt

# i denna cell kommer vi att skapa meningar som berättar vad som hände i fallet, vi förbredder detta för lokal maskin ska lebela de med vår utvalda ord 

In [5]:
import pandas as pd
import spacy
import re

nlp = spacy.load("xx_ent_wiki_sm")
nlp.add_pipe("sentencizer")


def extract_belopp(text):
    pattern = r'(\d[\d\s\u00a0\u202f]*)(?=\s*(kr|kronor|sek))'
    matches = re.findall(pattern, text.lower())
    # Ta bort alla typer av mellanslag
    return [int(re.sub(r'[\s\u00a0\u202f]', '', m[0])) for m in matches if m[0]]

def clean_summary(sent, belopp):
    subj = ""
    obj = ""
    for token in sent:
        if token.dep_ in ("nsubj", "nsubj:pass") and not subj:
            subj = token.text
        elif token.dep_ in ("dobj", "obj", "iobj") and not obj:
            obj = token.text
    if subj and obj:
        return f"{subj} ska betala {belopp[0]} kr till {obj}"
    elif subj:
        return f"{subj} ska betala {belopp[0]} kr"
    else:
        return sent.text.strip()

results = []
seen_ids = set()

# Lägg till felhantering för debugging
for idx, (_, row) in enumerate(df.iterrows()):
    try:
        all_text = ' '.join(str(row[col]) for col in df.columns)
        doc = nlp(all_text)
        for sent in doc.sents:
            if len(sent.text.split()) > 40:
                continue
            text = sent.text.lower()
            if any(term in text for term in ['kr', 'sek', 'skadebelopp']):
                belopp = extract_belopp(sent.text)
                if belopp:
                    summary = clean_summary(sent, belopp)
                    uid = f"{row['Id']}_{summary}"
                    if uid not in seen_ids:
                        seen_ids.add(uid)
                        results.append({
                            "id": row["Id"],
                            "rubrik": row["Rubrik"],
                            "utfall": row["Utfall"],
                            "kort_mening": summary,
                            "belopp": belopp
                        })
    except Exception as e:
        print(f"Fel vid rad {idx}: {e}")
        # Valfritt: Skriv ut problematisk text för att hjälpa med debugging
        if 'sent' in locals():
            print(f"Problematisk mening: {sent.text}")

print(f"Hittade {len(results)} korta, begripliga meningar.")

# Spara som CSV istället för JSON
output_df = pd.DataFrame(results)
output_df.to_csv("adjust_data_big_data.csv", index=False, encoding="utf-8")

Hittade 26680 korta, begripliga meningar.
